In [61]:
import os
import pandas as pd

In [62]:
names_db_path = "/Volumes/TBHD_share/DATABASES/names.dmp"

df = pd.read_csv(names_db_path, sep="|", header=None, names=["tax_id", "name_txt", "unique_name", "name_class", "blank"], dtype={"tax_id": str, "name_txt": str, "unique_name": str, "name_class": str, "blank": str})
df.drop("blank", axis=1, inplace=True)
df.head()

,tax_id,name_txt,unique_name,name_class
0,1\t,\tall\t,\t\t,\tsynonym\t
1,1\t,\troot\t,\t\t,\tscientific name\t
2,2\t,\tBacteria\t,\tBacteria <bacteria>\t,\tscientific name\t
3,2\t,\tbacteria\t,\t\t,\tblast name\t
4,2\t,\teubacteria\t,\t\t,\tgenbank common name\t


In [63]:
# Remove the \t character from all columns.
df = df.apply(lambda x: x.str.strip())
df.head()

,tax_id,name_txt,unique_name,name_class
0,1,all,,synonym
1,1,root,,scientific name
2,2,Bacteria,Bacteria <bacteria>,scientific name
3,2,bacteria,,blast name
4,2,eubacteria,,genbank common name


In [64]:
example_names = ["Escherichia coli", "E. coli", "Acetivibrio thermocellus"]

In [70]:
from typing import List

# Find the tax_id for the example names.
def find_tax_id(name_list: List[str]) -> List:
    names_dict = {}
    for name in name_list:
        try:
            names_dict[name] = df[df["name_txt"] == name]["tax_id"].values[0]
        except IndexError:
            names_dict[name] = None

    return names_dict

def find_names_from_tax_ids(tax_ids: List[str]) -> List:
    tax_id_names_dict = {}

    for tax_id in tax_ids:
        try:
            tax_id_names_dict[tax_id] = list(df[df["tax_id"] == tax_id]["name_txt"].values)
        except IndexError:
            tax_id_names_dict[tax_id] = None

    return tax_id_names_dict

### Using the Expected DataFrame for conversion to TaxIDs.

In [66]:
species_df_path = "bmock12_names.csv"

b12 = pd.read_csv(species_df_path, sep=",", dtype={"species": str, "tax_id": str}, index_col=0)
b12.head()

,ILLUMINA
Organism,
Muricauda sp. ES.050,74263482
Thioclava sp. ES.032,45769662
Cohaesibacter sp. ES.047,64862842
Propionibacteriaceae bacterium ES.041,20090318
Marinobacter sp. LV10R510-8,29119402


In [72]:
res = find_tax_id(b12.index.tolist())

# Add the tax_id to the dataframe.
b12["tax_id"] = b12.index.map(res)
b12

,ILLUMINA,tax_id
Organism,,
Muricauda sp. ES.050,74263482,1798204
Thioclava sp. ES.032,45769662,1798206
Cohaesibacter sp. ES.047,64862842,1798205
Propionibacteriaceae bacterium ES.041,20090318,1945889
Marinobacter sp. LV10R510-8,29119402,1415571
Marinobacter sp. LV10MA510-1,71778728,1415567
Psychrobacter sp. LV10R520-6,80284678,1415574
Micromonospora echinaurantiaca,12987850,47857
Micromonospora echinofusca,4794346,47858
